In [1]:
from __future__ import division
import random
import itertools

beat = {'R': 'P', 'P': 'S', 'S': 'R'}

In [12]:
class MarkovChain():
    def __init__(self, type, beat, level, memory, score=0, score_mem=0.9):
        self.type = type
        self.matrix = self.create_matrix(beat, level, memory)
        self.memory = memory
        self.level = level
        self.beat = beat
        self.score = score
        self.score_mem = score_mem
        self.prediction = ''
        self.name = 'level: {}, memory: {}'.format(self.level, self.memory)
        self.last_updated_key = ''

    @staticmethod
    def create_matrix(beat, level, memory):
        def create_keys(beat, level):
            keys = list(beat)

            if level > 1:

                for i in range(level - 1):
                    key_len = len(keys)
                    for i in itertools.product(keys, ''.join(beat)):
                        keys.append(''.join(i))
                    keys = keys[key_len:]

            return keys

        keys = create_keys(beat, level)

        matrix = {}
        for key in keys:
            matrix[key] = {'R': 1 / (1 - memory) / 3,
                           'P': 1 / (1 - memory) / 3,
                           'S': 1 / (1 - memory) / 3}
                           
        return matrix

    def update_matrix(self, key_lagged, response):

        for key in self.matrix[key_lagged]:
            self.matrix[key_lagged][key] = self.memory * self.matrix[key_lagged][key]

        self.matrix[key_lagged][response] += 1
        self.last_updated_key = key_lagged

    def update_score(self, inp, out):

        if self.beat[out] == inp:
            self.score = self.score * self.score_mem - 1
        elif out == inp:
            self.score = self.score * self.score_mem
        else:
            self.score = self.score * self.score_mem + 1

    def predict(self, key_current):

        probs = self.matrix[key_current]

        if max(probs.values()) == min(probs.values()):
            self.prediction = random.choice(list(beat.keys()))
        else:
            self.prediction = max([(i[1], i[0]) for i in probs.items()])[1]

        if self.type == 'input_oriented':
            return self.prediction
        elif self.type == 'output_oriented':
            return self.beat[self.prediction]

In [25]:
a = MarkovChain('input_oriented', beat, 1, 0.8)

In [26]:
a.matrix

{'R': {'R': 1.666666666666667, 'P': 1.666666666666667, 'S': 1.666666666666667},
 'P': {'R': 1.666666666666667, 'P': 1.666666666666667, 'S': 1.666666666666667},
 'S': {'R': 1.666666666666667, 'P': 1.666666666666667, 'S': 1.666666666666667}}

In [4]:
memory = [0.5, 0.6, 0.7, 0.8, 0.9, 0.93, 0.95, 0.97, 0.99]
level = [1, 2, 3, 4]

In [5]:
models_inp = [MarkovChain('input_oriented', beat, i[0], i[1]) for i in itertools.product(level, memory)]

In [14]:
for i in itertools.product(level, memory):
    print(i)

(1, 0.5)
(1, 0.6)
(1, 0.7)
(1, 0.8)
(1, 0.9)
(1, 0.93)
(1, 0.95)
(1, 0.97)
(1, 0.99)
(2, 0.5)
(2, 0.6)
(2, 0.7)
(2, 0.8)
(2, 0.9)
(2, 0.93)
(2, 0.95)
(2, 0.97)
(2, 0.99)
(3, 0.5)
(3, 0.6)
(3, 0.7)
(3, 0.8)
(3, 0.9)
(3, 0.93)
(3, 0.95)
(3, 0.97)
(3, 0.99)
(4, 0.5)
(4, 0.6)
(4, 0.7)
(4, 0.8)
(4, 0.9)
(4, 0.93)
(4, 0.95)
(4, 0.97)
(4, 0.99)


In [9]:
memory = [0.5, 0.6, 0.7, 0.8, 0.9, 0.93, 0.95, 0.97, 0.99]
level = [1, 2, 3, 4]
ensemble_min_score = [5]

models_inp = [MarkovChain('input_oriented', beat, i[0], i[1]) for i in itertools.product(level, memory)]
models_out = [MarkovChain('output_oriented', beat, i[0], i[1]) for i in itertools.product(level, memory)]

In [11]:
models_inp + models_out

In [27]:
class HistoryColl():
    def __init__(self):
        self.history = ''

    def hist_collector(self, inp, out):
        self.history = self.history + inp
        self.history = self.history + out
        if len(self.history) > 10:
            self.history = self.history[-10:]

    def create_keys(self, level):
        return self.history[-level:]

    def create_keys_hist(self, level):
        key_hist = self.history[-level - 2:-2]
        inp_latest = self.history[-2]
        out_latest = self.history[-1]
        return key_hist, inp_latest, out_latest

In [28]:
h = HistoryColl()

In [47]:
h.hist_collector('P', 'R')

In [45]:
h.hist_collector('R', 'S')

In [48]:
h.history

'PRRSPRRSPR'

In [53]:
h.create_keys(2)

'PR'

In [56]:
h.create_keys_hist(5)

('SPRRS', 'P', 'R')